In [ ]:
### FOR COLAB
import os
is_colab = 'google.colab' in str(get_ipython())
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    colab_path = '/content/drive/MyDrive/HumbugDB new/HumBugDB/notebooks'
    if not os.getcwd().endswith('ish'):
      os.chdir(colab_path)
    ## DO VERSION CORRECTING STUFFX_test.shape
    
    # TODO
import sys
sys.path.insert(0, os.path.abspath('../torchvggish/torchvggish'))
sys.path.append('../lib')

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import config
from sklearn.utils import shuffle, class_weight
from sklearn.preprocessing import scale
from tqdm import tqdm
import datetime
import pickle
from vggish_input import wavfile_to_examples
from os.path import join
from evaluate import get_results, plot_confusion_matrix_multiclass, compute_plot_roc_multiclass

In [ ]:
model = torch.hub.load('harritaylor/torchvggish', 'vggish')
model.eval()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Download an example audio file
import urllib
url, filename = ("http://soundbible.com/grab.php?id=1698&type=wav", "bus_chatter.wav")
file_dir = '../torchvggish'
try: urllib.URLopener().retrieve(url, os.path.join(file_dir, filename))
except: urllib.request.urlretrieve(url, os.path.join(file_dir, filename))

bus_chatter_out = model.forward(os.path.join(file_dir, filename))

In [ ]:
bus_chatter_out = model.forward(os.path.join(file_dir, filename))
bus_chatter_out

In [ ]:
log_mel_examples = wavfile_to_examples(join(file_dir, 'test.wav')) 
bus_feat = wavfile_to_examples(join(file_dir, 'bus_chatter.wav')) 

In [ ]:
print(log_mel_examples.shape, bus_feat.shape)

In [ ]:
print(model)

# Dataframe stuff


In [ ]:
model.preprocess = False # we can't use the standard preprocessing as we need to work from clips

In [ ]:
df = pd.read_csv(config.data_df)
idx_multiclass = np.logical_and(df['country'] == 'Tanzania', df['location_type'] == 'cup')
df_all = df[idx_multiclass]
classes = ['an arabiensis','culex pipiens complex', 'ae aegypti','an funestus ss','an squamosus',
               'an coustani','ma uniformis','ma africanus']

In [ ]:
# work with much smaller subset of data so this runs quicker - get rid of once it's working
# and replace with a pckle saver/loader
# df_all = df_all.iloc[::20,:]

In [ ]:
# copied from species_classification
import collections
species_dict = collections.OrderedDict()
species_recordings = collections.OrderedDict()
for species in classes:
    # Number of total audio clips per species (includes repeats from same filename)
    species_recordings[species] = len(pd.unique(df_all[df_all.species==species].name)) # Number of unique audio recordings (and hence mosquitoes)
    species_dict[species] = sum(df_all[df_all.species==species].length)

In [ ]:
# Divide recordings into train and test, with recording shuffling fixed by random_state
train_fraction = 0.75
train_recordings = {}
test_recordings = {}

for i in range(len(classes)):
    n_train = int(species_recordings[classes[i]] * train_fraction)
    n_test = species_recordings[classes[i]] - n_train
    print(classes[i], n_train, n_test)
    df_class = df_all[df_all.species == classes[i]]
    # !!!I think it is probably best to shuffle these to mimic W but need to make sure each ID of mosquito is unique per recording
    train_recordings[i] =  shuffle(pd.unique(df_class.name), random_state=42)[:n_train]  
    test_recordings[i] = shuffle(pd.unique(df_class.name),random_state=42)[n_train:]

# Make and pickle features

In [ ]:
def make_vggish_feats(df_all, label_recordings_dict, data_dir):
  X, y = [], []
  for class_label in tqdm(label_recordings_dict.keys(), position=0, leave=True): # Loop over classes
    for i in label_recordings_dict[class_label]: # Loop over recordings in class
      df_match = df_all[df_all.name == i]
      for idx, row in df_match.iterrows(): # Loop over clips in recording
        _, file_format = os.path.splitext(row['name'])
        filename = os.path.join(data_dir, str(row['id']) + file_format)
        feat = wavfile_to_examples(filename)
        # if config.norm_per_sample:
          # feat = (feat-np.mean(feat))/np.std(feat) # TODO come back to          
        X.append(feat)
        y.append(class_label)
  return X, y


In [ ]:
# X_test, y_test = make_vggish_feats(df_all, test_recordings, config.data_dir)

In [ ]:
# X_train, y_train = make_vggish_feats(df_all, train_recordings, config.data_dir)

In [ ]:
# Now we need a reshaper to turn this into a form that the network can accept
# we have a list of n tensors and n labels, each tensor mx1x96x64
# we need to produce a single training tensor that is _x1x96x64

In [ ]:
def reshape(X, y):
  tmp = []
  for i in range(len(X)):
    tmp += [y[i]] * X[i].shape[0]
  y = np.array(tmp)
  X = torch.cat(X, dim=0)
  return X, y

In [ ]:
# X_train, y_train = reshape(X_train, y_train)
# X_test, y_test = reshape(X_test, y_test)

In [ ]:
def pickle_features():
  # Pickle features for whole dataset
  dt = datetime.datetime.now()
  seq = str(int(dt.strftime("%Y%m%d%H%M%S")))
  pickle_dir='../outputs/features/vggish'
  if not os.path.exists(pickle_dir):
    os.mkdir(pickle_dir)
  test_pickle = 'test_feats_vggish_' + seq + '.pkl'
  train_pickle = 'train_feats_vggish_' + seq + '.pkl'
  with open(join(pickle_dir, train_pickle), 'wb') as f:
    pickle.dump([X_train, y_train], f, protocol=4)
  with open(join(pickle_dir, test_pickle), 'wb') as f:
    pickle.dump([X_test, y_test], f, protocol=4)
# pickle_features()

# Load pickled features

In [ ]:
def load_vggish_feats(file, dir='../outputs/features/vggish'):
  if os.path.exists(join(dir, file)):
    with open(join(dir, file), 'rb') as f:
      [X, y] = pickle.load(f)
  return X, y

In [ ]:
test_file = 'test_feats_vggish_20210802124327.pkl'
train_file = 'train_feats_vggish_20210802124327.pkl'
X_train, y_train = load_vggish_feats(train_file)
X_test, y_test = load_vggish_feats(test_file)

In [ ]:
print(len(X_train), X_train[1].shape, len(y_train), type(y_train[0]))

In [ ]:
from runTorchMultiClass import train_model, test_model, load_model, evaluate_model
# test_X_train = test_feat_output # as in from test.wav not tes data
# test_y_train = np.random.randint(8, size=(4,)) # numpy array not torch tensor - a problem?

In [ ]:
X_train_embed = model.forward(X_train[280:560])
alist = [X_train_embed]*100

In [ ]:
X_train_embed.shape

In [ ]:
model.preprocessing = False
X_train_embed = []
for i in range(101):
  chunk = [280*i, 280*(i+1)]
  print(chunk)
  X_train_embed.append(model.forward(X_train[chunk[0]:chunk[1]]))
len(X_train_embed)


In [ ]:
100*[X_train_embed]

In [ ]:
model.preprocessing = False
X_train_embed = []
for i in range(6):
  chunk = [280*i, 280*(i+1)]
  print(chunk)
  X_train_embed.append(model.forward(X_train[chunk[0]:chunk[1]]))
len(X_train_embed)

# Train model

In [ ]:
num_classes = 8 # in a config file or something
class VGGishMulticlass(nn.Module):
  def __init__(self, num_classes, vggish_model=None, dropout=0.2): # dropout?
    super(VGGishMulticlass, self).__init__()
    self.dropout = dropout
    # TODO go back to how it was before, so you can control preprocessing
    if vggish_model is not None:
      self.vggish_embedding = vggish_model
    else: 
      self.vggish_embedding = torch.hub.load('harritaylor/torchvggish', 'vggish', pretrained=True)
    self.fc1 = nn.Linear(128, num_classes)
  def forward(self, x):
      x = self.vggish_embedding(x).squeeze()
      x = x/255   # normalise fc1 input between 0 and 1
      # x_mean, x_std = torch.mean(x, axis=1), torch.std(x, axis=1)
      # x = x - x_mean.reshape(-1,1)
      # x = x / x_std.reshape(-1,1)   # Z-normalise inputs to fc1
      
      # print(x.shape)
      x = self.fc1(F.dropout(x,p=self.dropout))
      # x = torch.sigmoid(x) 
      # print(x[0])
      # x = F.relu(x)
      
      return x

In [ ]:
model = torch.hub.load('harritaylor/torchvggish', 'vggish', pretrained=True)
model.eval()
model.preprocess = False
print(model(X_train[:10]))
print(model.forward(X_train[:10]))

In [ ]:
# Testing class
x = VGGishMulticlass(num_classes)
x = x.to(device)
x.preprocess = True
# x.forward(join(file_dir, 'bus_chatter.wav'))[0]

In [ ]:
model = torch.hub.load('harritaylor/torchvggish', 'vggish')
model.preprocess = False
model.postprocess = False
class_weights = class_weight.compute_class_weight('balanced',
                                                  classes=np.unique(y_train),
                                                  y=y_train)
trained_model = train_model(X_train, y_train, 
                            class_weight=class_weights, 
                            model=VGGishMulticlass(num_classes, model))

In [ ]:
sample_y_pred = [ 86.7832,  73.5336,  13.5915,  73.2701,  26.7167,  67.8825,  19.7151, 10.0495]
import scipy
print(scipy.special.softmax(sample_y_pred))
print(np.log(scipy.special.softmax(sample_y_pred)))

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
trained_model.forward(X_test)

In [ ]:
evaluate_model(trained_model, X_test, y_test, n_samples=10)

# Experimenting/bug fixing

## Plotting embeddings

In [ ]:
def plot_embeddings(tensor, labels):
  plt.figure(figsize=[25,8])
  plt.plot(tensor.detach().numpy().T)
  labels = labels.tolist()
  for i in range(len(labels)):
    labels[i]= str(labels[i])
  print(labels)
  plt.legend(labels)

In [ ]:
# plot embedded features from standard model
model = torch.hub.load('harritaylor/torchvggish', 'vggish')
model.postprocess = False
model.preprocess = False
random_sample = np.random.randint(X_test.shape[0], size=(5,))
print(random_sample)
X_test_fraction = X_test[random_sample]
y_labels_sample = y_test[random_sample]
# X_test_fraction = X_test[np.arange(4)*3000]
output = model.forward(X_test_fraction).cpu()
plot_embeddings(output, y_labels_sample)

In [ ]:
X_test.shape

In [ ]:
# compare with features from bus_chatter.wav
plot_embeddings(bus_chatter_out[:5].cpu())

## Checking values before going into loss function/softmax